In [1]:
import pandas as pd
import numpy as np
import os
from IPython.display import display, HTML

In [2]:
pd.options.display.max_columns = 30
df = pd.read_csv('../data/csl/CSL_d5_targets_2020-08-21.csv', index_col=0)
print(df.shape)
display(df.head())

(185413, 250)


,Sitenum,Momdeath,MomICU,MomNearMiss,Bloodproduct,Posttransfus,transfus_yes,EBLoss,high_EBLoss,trans_loss,Hysterectomy,transfus_all,transfus_hyster,AB_any,AB_ceph,...,Woundinf,Woundsep,Activeherpes,ga,hie_new1,ich_new1,iufd9,myocardio_new1,R_preg,Outcome1,high_BMI,high_Age,high_Gravidity,high_MomLOS,high_height
MomID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
41-00002,41,1,1,1,1,1,1,350.0,0,1,1,1,1,1,1,...,1,1,1,38,1,1,0,1,1,1,0,0,2,3,2.0
41-00003,41,1,1,1,1,1,1,250.0,0,1,1,1,1,1,1,...,1,1,1,39,1,1,0,1,1,1,0,0,3,2,2.0
41-00004,41,1,1,1,1,1,1,250.0,0,1,1,1,1,1,1,...,1,1,1,39,1,1,0,1,1,1,0,0,4,1,3.0
41-00005,41,1,1,1,1,2,2,350.0,0,2,1,2,2,1,1,...,1,1,1,38,1,1,0,1,1,1,0,0,3,2,2.0
41-00006,41,1,1,1,1,1,1,450.0,0,1,1,1,1,1,1,...,1,1,1,39,1,1,0,1,1,1,0,0,1,2,3.0


In [3]:
# Create target datasets that don't have dependent vars. or other targets

# transfus_yes depends on 'Bloodproduct' and 'Posttransfus'
# MomNearMiss, trans_loss, transfus_all, and transfus_hyster are derived variables
df.drop('Bloodproduct', axis=1, inplace=True)
df.drop('Posttransfus', axis=1, inplace=True)
df.drop('MomNearMiss', axis=1, inplace=True)
df.drop('trans_loss', axis=1, inplace=True)
df.drop('transfus_all', axis=1, inplace=True)
df.drop('transfus_hyster', axis=1, inplace=True)

# Also remove numerous collinear and possibly confounding variables
df.drop('MomICU', axis=1, inplace=True) # New - 06Nov2020
df.drop('EBLoss', axis=1, inplace=True)
df.drop('high_EBLoss', axis=1, inplace=True)
df.drop('hemorrhage', axis=1, inplace=True)
df.drop('postpartum_hemorrhage9', axis=1, inplace=True)
df.drop('Postbleed', axis=1, inplace=True)
df.drop('high_MomLOS', axis=1, inplace=True)
df.drop('FSE', axis=1, inplace=True)
df.drop('GAmethod', axis=1, inplace=True)
df.drop('Ligation', axis=1, inplace=True)
df.drop('Sitenum', axis=1, inplace=True)
df.drop('Analgesia', axis=1, inplace=True)
df.drop('fever9', axis=1, inplace=True)
df.head()

,Momdeath,transfus_yes,Hysterectomy,AB_any,AB_ceph,AB_clind,AB_doxy,AB_eryth,AB_flagyl,AB_gent,AB_peni,AB_vanc,abruptio9,abruption,Accrete,...,vertex,Woundinf,Woundsep,Activeherpes,ga,hie_new1,ich_new1,iufd9,myocardio_new1,R_preg,Outcome1,high_BMI,high_Age,high_Gravidity,high_height
MomID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
41-00002,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,...,9,1,1,1,38,1,1,0,1,1,1,0,0,2,2.0
41-00003,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,...,1,1,1,1,39,1,1,0,1,1,1,0,0,3,2.0
41-00004,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,...,1,1,1,1,39,1,1,0,1,1,1,0,0,4,3.0
41-00005,1,2,1,1,1,1,1,1,1,1,1,1,0,1,1,...,1,1,1,1,38,1,1,0,1,1,1,0,0,3,2.0
41-00006,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,...,1,1,1,1,39,1,1,0,1,1,1,0,0,1,3.0


In [4]:
filename = '../data/csl/CSL_d5_transfus_yes_2020-11-09.csv'
if not os.path.exists(filename):  # Comment out this line to overwrite file
    df.to_csv(filename, header=True)

In [5]:
df = pd.read_csv('../data/csl/CSL_d5_targets_2020-08-21.csv', index_col=0)
print(df.shape)

# trans_loss depends on transfus_yes so all its variables need removing, too
df.drop('Bloodproduct', axis=1, inplace=True)
df.drop('Posttransfus', axis=1, inplace=True)
df.drop('MomNearMiss', axis=1, inplace=True)
df.drop('transfus_yes', axis=1, inplace=True)
df.drop('transfus_all', axis=1, inplace=True)
df.drop('transfus_hyster', axis=1, inplace=True)

# Also remove numerous collinear and possibly confounding variables
df.drop('MomICU', axis=1, inplace=True) # New - 06Nov2020
df.drop('EBLoss', axis=1, inplace=True)
df.drop('high_EBLoss', axis=1, inplace=True)
df.drop('hemorrhage', axis=1, inplace=True)
df.drop('postpartum_hemorrhage9', axis=1, inplace=True)
df.drop('Postbleed', axis=1, inplace=True)
df.drop('high_MomLOS', axis=1, inplace=True)
df.drop('FSE', axis=1, inplace=True)
df.drop('GAmethod', axis=1, inplace=True)
df.drop('Ligation', axis=1, inplace=True)
df.drop('Sitenum', axis=1, inplace=True)
df.drop('Analgesia', axis=1, inplace=True)
df.drop('fever9', axis=1, inplace=True)
df.head()

(185413, 250)


,Momdeath,trans_loss,Hysterectomy,AB_any,AB_ceph,AB_clind,AB_doxy,AB_eryth,AB_flagyl,AB_gent,AB_peni,AB_vanc,abruptio9,abruption,Accrete,...,vertex,Woundinf,Woundsep,Activeherpes,ga,hie_new1,ich_new1,iufd9,myocardio_new1,R_preg,Outcome1,high_BMI,high_Age,high_Gravidity,high_height
MomID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
41-00002,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,...,9,1,1,1,38,1,1,0,1,1,1,0,0,2,2.0
41-00003,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,...,1,1,1,1,39,1,1,0,1,1,1,0,0,3,2.0
41-00004,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,...,1,1,1,1,39,1,1,0,1,1,1,0,0,4,3.0
41-00005,1,2,1,1,1,1,1,1,1,1,1,1,0,1,1,...,1,1,1,1,38,1,1,0,1,1,1,0,0,3,2.0
41-00006,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,...,1,1,1,1,39,1,1,0,1,1,1,0,0,1,3.0


In [6]:
filename = '../data/csl/CSL_d5_trans_loss_2020-11-09.csv'
#if not os.path.exists(filename):  # Comment out this line to overwrite file
df.to_csv(filename, header=True)

In [7]:
df = pd.read_csv('../data/csl/CSL_d5_targets_2020-08-21.csv', index_col=0)
print(df.shape)
display(df.head())
# Create target datasets that don't have dependent vars. or other targets

# transfus_hyster depends on transfus_yes and hysterectomy
df.drop('Bloodproduct', axis=1, inplace=True)
df.drop('Posttransfus', axis=1, inplace=True)
df.drop('MomNearMiss', axis=1, inplace=True)
df.drop('Hysterectomy', axis=1, inplace=True)
df.drop('transfus_yes', axis=1, inplace=True)
df.drop('trans_loss', axis=1, inplace=True)
df.drop('transfus_all', axis=1, inplace=True)

# Also remove numerous collinear and possibly confounding variables
df.drop('MomICU', axis=1, inplace=True) # New - 06Nov2020
df.drop('EBLoss', axis=1, inplace=True)
df.drop('high_EBLoss', axis=1, inplace=True)
df.drop('hemorrhage', axis=1, inplace=True)
df.drop('postpartum_hemorrhage9', axis=1, inplace=True)
df.drop('Postbleed', axis=1, inplace=True)
df.drop('high_MomLOS', axis=1, inplace=True)
df.drop('FSE', axis=1, inplace=True)
df.drop('GAmethod', axis=1, inplace=True)
df.drop('Ligation', axis=1, inplace=True)
df.drop('Sitenum', axis=1, inplace=True)
df.drop('Analgesia', axis=1, inplace=True)
df.drop('fever9', axis=1, inplace=True)
df.head()

(185413, 250)


,Sitenum,Momdeath,MomICU,MomNearMiss,Bloodproduct,Posttransfus,transfus_yes,EBLoss,high_EBLoss,trans_loss,Hysterectomy,transfus_all,transfus_hyster,AB_any,AB_ceph,...,Woundinf,Woundsep,Activeherpes,ga,hie_new1,ich_new1,iufd9,myocardio_new1,R_preg,Outcome1,high_BMI,high_Age,high_Gravidity,high_MomLOS,high_height
MomID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
41-00002,41,1,1,1,1,1,1,350.0,0,1,1,1,1,1,1,...,1,1,1,38,1,1,0,1,1,1,0,0,2,3,2.0
41-00003,41,1,1,1,1,1,1,250.0,0,1,1,1,1,1,1,...,1,1,1,39,1,1,0,1,1,1,0,0,3,2,2.0
41-00004,41,1,1,1,1,1,1,250.0,0,1,1,1,1,1,1,...,1,1,1,39,1,1,0,1,1,1,0,0,4,1,3.0
41-00005,41,1,1,1,1,2,2,350.0,0,2,1,2,2,1,1,...,1,1,1,38,1,1,0,1,1,1,0,0,3,2,2.0
41-00006,41,1,1,1,1,1,1,450.0,0,1,1,1,1,1,1,...,1,1,1,39,1,1,0,1,1,1,0,0,1,2,3.0


,Momdeath,transfus_hyster,AB_any,AB_ceph,AB_clind,AB_doxy,AB_eryth,AB_flagyl,AB_gent,AB_peni,AB_vanc,abruptio9,abruption,Accrete,AdmBishop,...,vertex,Woundinf,Woundsep,Activeherpes,ga,hie_new1,ich_new1,iufd9,myocardio_new1,R_preg,Outcome1,high_BMI,high_Age,high_Gravidity,high_height
MomID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
41-00002,1,1,1,1,1,1,1,1,1,1,1,0,1,1,8,...,9,1,1,1,38,1,1,0,1,1,1,0,0,2,2.0
41-00003,1,1,1,1,1,1,1,1,1,1,1,0,1,1,6,...,1,1,1,1,39,1,1,0,1,1,1,0,0,3,2.0
41-00004,1,1,1,1,1,1,1,1,1,1,1,0,1,1,5,...,1,1,1,1,39,1,1,0,1,1,1,0,0,4,3.0
41-00005,1,2,1,1,1,1,1,1,1,1,1,0,1,1,11,...,1,1,1,1,38,1,1,0,1,1,1,0,0,3,2.0
41-00006,1,1,1,1,1,1,1,1,1,1,1,0,1,1,11,...,1,1,1,1,39,1,1,0,1,1,1,0,0,1,3.0


In [8]:
filename = '../data/csl/CSL_d5_transfus_hyster_2020-11-09.csv'
if not os.path.exists(filename): # Comment out this line to overwrite file
    df.to_csv(filename, header=True)